In [45]:
import pymongo, csv
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient()
pd.options.display.max_rows = 999

In [46]:
grade_map_path = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/6_app/grade_map.csv'
grade_num_map_path = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/6_app/grade_num_map.csv'
write_loc = '/Users/colinbrochard/DSI_Capstone_local/MtProjRec/2_data/5_route_features/routes_features.csv'

In [47]:
# pull data from Mongo DB into pd DF
db = client.routesAPI_db
df = pd.DataFrame(list(db.routesAPI_db.find()))

In [48]:
# clean up ratings
with open(grade_map_path) as f:
    data = [line.strip().split("|") for line in f.readlines()]
grade_map = {d[0]: d[1] for d in data}
df['rating0'] = df['rating'].map(grade_map)

In [49]:
# map ratings to number
with open(grade_num_map_path) as f:
    data = [line.strip().split("|") for line in f.readlines()]
grade_num_map = {d[0]: d[1] for d in data}
df['rating_num0'] = df['rating0'].map(grade_num_map)

In [50]:
df['location0'] = df['location'].apply(lambda x: x[0])

In [51]:
conditions_t = [
    (df['type'] == 'Sport') | (df['type'] == 'Sport, Aid'),
    (df['type'] == 'Boulder'),
    (df['type'] == 'Trad')]
choices_t = ['Sport', 'Boulder', 'Trad']
df['type0'] = np.select(conditions_t, choices_t, default='Trad')

In [52]:
conditions_p = [(df['pitches'] == 1) | (df['pitches'] == '')]
choices_p = ['single-pitch']
df['pitches0'] = np.select(conditions_p, choices_p, default='multi-pitch')

In [53]:
df_features = df[['id','name','type0','rating0','rating_num0','pitches0','location0']]

In [54]:
df_features.to_csv(write_loc, sep='|')

In [44]:
df_features[df_features['rating_num0'].isnull()].shape

(25, 7)

In [42]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86898 entries, 0 to 86897
Data columns (total 7 columns):
id             86898 non-null int64
name           86898 non-null object
type0          86898 non-null object
rating0        86873 non-null object
rating_num0    86873 non-null object
pitches0       86898 non-null object
location0      86898 non-null object
dtypes: int64(1), object(6)
memory usage: 4.6+ MB
